# Data Preparation - IT2 - Choosing best flow 

In [16]:
import os
import random
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm


In [17]:
# Define the paths
folder_path = '../data/original'  # Update this path to point to your specific folder containing images
output_folder = '../data/processed'  # Update this path to the desired output folder


## Loading Images and stats

In [18]:
# Load image paths
def load_images_from_folder(folder_path, extensions=('.png', '.jpg', '.jpeg', '.JPG')):
    """
    Load all image file paths from a specified folder that match the given file extensions.

    Parameters:
    folder_path (str): The path to the folder containing the images.
    extensions (tuple of str): A tuple of file extensions to filter the images by. 
                               Default is ('.png', '.jpg', '.jpeg', '.JPG').

    Returns:
    list: A list of full file paths to images in the folder that match the specified extensions.
    
    Raises:
    FileNotFoundError: If the specified folder does not exist.
    """

    # Check if the folder exists
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"The specified folder does not exist: {folder_path}")

    # List comprehension to gather all image paths with the specified extensions
    image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(extensions)]

    return image_paths


In [19]:
# Load and preprocess images
def load_and_preprocess_images(image_paths, resize_dim=(256, 256)):
    images = []
    image_ids = []

    # Initialize tqdm progress bar
    for path in tqdm(image_paths, desc="Loading and preprocessing images", unit="image"):
        img = cv2.imread(path)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        img_resized = cv2.resize(img_gray, resize_dim)  # Resize for consistency
        images.append(img_resized)
        image_ids.append(f'Image_{len(images)}')  # Assign image ID as Image_1, Image_2, etc.

    return images, image_ids


In [20]:
# Load all image file paths from the specified folder
image_paths_all = load_images_from_folder(folder_path)

# Load and preprocess all images
total_images, total_image_ids = load_and_preprocess_images(image_paths_all)

# Randomly select 10 images for experimentation
experiment_indices = random.sample(range(len(total_images)), 10)
test_images = [total_images[i] for i in experiment_indices]
test_image_ids = [total_image_ids[i] for i in experiment_indices]


Loading and preprocessing images: 100%|██████████| 698/698 [02:41<00:00,  4.32image/s]


In [21]:
# Load the CSV file with the image statistics
images_stats_path = "../data-understanding/images_stats.csv"
images_stats_df = pd.read_csv(images_stats_path)


## Functions of Characteristics

In [22]:
# Image Characteristics Calculation Functions - from data understanding it2
def calculate_brightness(image):
    return np.mean(image)

def calculate_sharpness(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

def calculate_contrast(image):
    return image.std()

def calculate_noise(image):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(image, (3, 3), 0)
    noise = cv2.absdiff(image, blurred)
    return np.var(noise)

# def calculate_skew(image):
#     if len(image.shape) != 2:
#         raise ValueError("Invalid image format. Image must be a 2D grayscale image.")
#     _, binary = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)
#     coords = np.column_stack(np.where(binary > 0))
#     if coords.size == 0:
#         return 0
#     angle = cv2.minAreaRect(coords)[-1]
#     if angle < -45:
#         angle = -(90 + angle)
#     else:
#         angle = -angle
#     if abs(angle) < 1e-2:
#         angle = 0
#     return round(angle, 2)
# 
# def calculate_line_spacing(image):
#     if len(image.shape) != 2:
#         raise ValueError("Invalid image format. Image must be a 2D grayscale image.")
# 
#     # Ensure the image is in the correct format for findContours
#     if image.dtype != np.uint8:
#         image = cv2.convertScaleAbs(image)
# 
#     _, binary = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)
#     contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     heights = [cv2.boundingRect(contour)[3] for contour in contours]
#     if len(heights) > 1:
#         line_spacing = np.mean(np.diff(sorted(heights)))
#     else:
#         line_spacing = 0
#     return line_spacing
# 
# 
# def detect_tables(image):
#     if len(image.shape) != 2:
#         raise ValueError("Invalid image format. Image must be a 2D grayscale image.")
# 
#     # Ensure the image is in the correct format for adaptiveThreshold
#     if image.dtype != np.uint8:
#         image = cv2.convertScaleAbs(image)
# 
#     # Apply binary threshold and adaptive threshold
#     _, binary = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)
#     binary = cv2.adaptiveThreshold(binary, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
# 
#     # Find contours in the image
#     contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     table_contours = [contour for contour in contours if cv2.contourArea(contour) > 1000]
# 
#     return len(table_contours)
# 
# 
# def calculate_resolution(image):
#     height, width = image.shape[:2]
#     return height * width
# 
# def calculate_elements_detection(image):
#     if len(image.shape) != 2:
#         raise ValueError("Invalid image format. Image must be a 2D grayscale image.")
# 
#     # Ensure the image is in the correct format for findContours
#     if image.dtype != np.uint8:
#         image = cv2.convertScaleAbs(image)
# 
#     # Apply binary threshold
#     _, binary = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)
# 
#     # Find contours
#     contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     return len(contours)
# 
# 
# def calculate_texture(image):
#     laplacian = cv2.Laplacian(image, cv2.CV_64F)
#     return laplacian.std()
# 
# def calculate_patterns(image):
#     if len(image.shape) != 2:
#         raise ValueError("Invalid image format. Image must be a 2D grayscale image.")
# 
#     # Ensure the image is in the correct format for Canny edge detection
#     if image.dtype != np.uint8:
#         image = cv2.convertScaleAbs(image)
# 
#     # Apply Canny edge detection
#     edges = cv2.Canny(image, 100, 200)
# 
#     # Count the number of edge pixels
#     return np.sum(edges > 0)



## Evaluation Function

In [23]:
def advanced_evaluation(image, techniques_dict, original_stats):
    evaluation_results = {}

    for technique_name, technique_func in techniques_dict.items():
        # Apply the technique
        processed_image = technique_func(image)

        # Calculate characteristics for the processed image
        stats = {
            "Brightness": calculate_brightness(processed_image),
            "Sharpness": calculate_sharpness(processed_image),
            "Contrast": calculate_contrast(processed_image),
            "Noise": calculate_noise(processed_image),
        }

        # Normalize metrics to comparable ranges (between 0 and 1, roughly)
        stats_normalized = {
            "Brightness": stats["Brightness"] / 255,
            "Sharpness": stats["Sharpness"] / 1000,
            "Contrast": stats["Contrast"] / 255,
            "Noise": stats["Noise"] / 255,
        }

        # Normalize the original stats for comparison
        original_stats_normalized = {
            "Brightness": original_stats["Brightness"] / 255,
            "Sharpness": original_stats["Sharpness"] / 1000,
            "Contrast": original_stats["Contrast"] / 255,
            "Noise": original_stats["Noise"] / 255,
        }

        # Weights for each characteristic (to determine their importance)
        weights = {
            "Brightness": -1.0,  # Closer to original is better (penalized if different) as they were in a good level before
            "Sharpness": 2.0,    # Higher is better (rewarded if improved)
            "Contrast": 1.0,     # Higher is better (rewarded if improved)
            "Noise": -1.5,       # Lower is better (penalized if increased)
        }

        # Calculate score using normalized metrics and weights
        score = 0
        for metric, value in stats_normalized.items():
            original_value = original_stats_normalized.get(metric, 0)
            score += weights[metric] * (value - original_value)

        evaluation_results[technique_name] = {"Score": score, "Stats": stats}

    # Determine the best technique based on the highest score
    best_technique = max(evaluation_results, key=lambda x: evaluation_results[x]["Score"])
    return {"Best Technique": best_technique, "Evaluation Results": evaluation_results}


## Setting the best techniques based on step 1.1

In [24]:
# Define the best techniques and parameters for each step
best_techniques = {
    "Noise Reduction": (lambda img: cv2.fastNlMeansDenoising(img, None, h=5, templateWindowSize=7, searchWindowSize=21)),
    "Histogram Equalization": (lambda img: cv2.equalizeHist(img)),
    "Binarization": (lambda img: cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, blockSize=11, C=2)),
    "Morphological Operations": (lambda img: cv2.erode(img, np.ones((3, 3), np.uint8))),
    "Edge Detection": (lambda img: cv2.Canny(img, threshold1=50, threshold2=150)),
}


## Using the previous flows found to be most suitable

In [25]:
# Best Flows from Previous Iteration
previous_best_flows = {
    "Flow - CLAHE": [
        ("Gaussian Blur", lambda img: cv2.GaussianBlur(img, (5, 5), 0)),
        ("CLAHE", lambda img: cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8)).apply(img))
    ]
}


## Setting flows based on best techniques

In [26]:
# Create New Flows Incrementally
new_flows = {
    "Flow 1: Noise Reduction": [
        ("Noise Reduction", best_techniques["Noise Reduction"])
    ],
    "Flow 2: Noise Reduction + Histogram Equalization": [
        ("Noise Reduction", best_techniques["Noise Reduction"]),
        ("Histogram Equalization", best_techniques["Histogram Equalization"])
    ],
    "Flow 3: Noise Reduction + Histogram Equalization + Binarization": [
        ("Noise Reduction", best_techniques["Noise Reduction"]),
        ("Histogram Equalization", best_techniques["Histogram Equalization"]),
        ("Binarization", best_techniques["Binarization"])
    ],
    "Flow 4: Noise Reduction + Histogram Equalization + Binarization + Morphological Operations": [
        ("Noise Reduction", best_techniques["Noise Reduction"]),
        ("Histogram Equalization", best_techniques["Histogram Equalization"]),
        ("Binarization", best_techniques["Binarization"]),
        ("Morphological Operations", best_techniques["Morphological Operations"])
    ],
    "Flow 5: Full Flow": [
        ("Noise Reduction", best_techniques["Noise Reduction"]),
        ("Histogram Equalization", best_techniques["Histogram Equalization"]),
        ("Binarization", best_techniques["Binarization"]),
        ("Morphological Operations", best_techniques["Morphological Operations"]),
        ("Edge Detection", best_techniques["Edge Detection"])
    ]
}


## Prepearing and running the flows

In [27]:
# Combine all flows
all_flows = {**previous_best_flows, **new_flows}

In [28]:
# Function to apply a flow to an image
def apply_flow(flow, image):
    for step_name, technique in flow:
        image = technique(image)
    return image


In [29]:
# Testing the Flows
flow_results = {}

# Loop over each flow
for flow_name, flow_steps in all_flows.items():
    total_score = 0
    print(f"Testing Flow: {flow_name}\n{'-' * 40}")

    # Loop over each image for evaluation
    for img, img_id in zip(test_images, test_image_ids):
        original_stats = images_stats_df[images_stats_df['Image'] == img_id].iloc[0].to_dict()

        # Apply the flow on the image
        processed_image = apply_flow(flow_steps, img)

        # Using advanced evaluation function to evaluate the processed image
        evaluation_results = advanced_evaluation(processed_image, {"Flow": lambda x: x}, original_stats)

        # Access the score for the flow (since only one technique is passed, we can directly fetch it)
        flow_score = evaluation_results["Evaluation Results"]["Flow"]["Score"]

        # Add the score to the total score
        total_score += flow_score
        print(f"Image {img_id}: Score = {flow_score}")

    # Calculate the average score for the flow
    avg_score = total_score / len(test_images)
    print(f"Average Score for Flow '{flow_name}': {avg_score}\n")
    flow_results[flow_name] = avg_score


Testing Flow: Flow - CLAHE
----------------------------------------
Image Image_338: Score = -0.7962729785216383
Image Image_372: Score = -2.2091320467620092
Image Image_539: Score = -3.394193565020206
Image Image_128: Score = -1.743632209612172
Image Image_173: Score = -1.5284510925833246
Image Image_41: Score = -1.5275381477795635
Image Image_690: Score = -2.404918095628553
Image Image_498: Score = -1.7921267088557724
Image Image_38: Score = -2.1785407206770686
Image Image_83: Score = -1.176229097051851
Average Score for Flow 'Flow - CLAHE': -1.8751034662492159

Testing Flow: Flow 1: Noise Reduction
----------------------------------------
Image Image_338: Score = -0.2547880676802674
Image Image_372: Score = -0.20983468310315936
Image Image_539: Score = -0.22989545345595758
Image Image_128: Score = -0.2427408631386082
Image Image_173: Score = -0.3012814414122185
Image Image_41: Score = -0.41469975947918564
Image Image_690: Score = -0.2869703949106332
Image Image_498: Score = -0.26015

In [30]:
# Display Final Results
sorted_results = sorted(flow_results.items(), key=lambda x: x[1], reverse=True)
print("\nFinal Flow Testing Results (Sorted by Score):")
for flow_name, score in sorted_results:
    print(f"{flow_name}: Average Score = {score}")


Final Flow Testing Results (Sorted by Score):
Flow 5: Full Flow: Average Score = 222.56925751968592
Flow 3: Noise Reduction + Histogram Equalization + Binarization: Average Score = 160.82440287704304
Flow 4: Noise Reduction + Histogram Equalization + Binarization + Morphological Operations: Average Score = 82.3969757191297
Flow 2: Noise Reduction + Histogram Equalization: Average Score = 4.074362102072227
Flow 1: Noise Reduction: Average Score = -0.29412017243243405
Flow - CLAHE: Average Score = -1.8751034662492159


Final Flow Testing Results (Sorted by Score):
Flow 5: Full Flow: Average Score = 222.57
Flow 3: Noise Reduction + Histogram Equalization + Binarization: Average Score = 160.82
Flow 4: Noise Reduction + Histogram Equalization + Binarization + Morphological Operations: Average Score = 82.40
Flow 2: Noise Reduction + Histogram Equalization: Average Score = 4.07
Flow 1: Noise Reduction: Average Score = -0.29
Flow - CLAHE: Average Score = -1.88

# Flow Testing Results and Interpretation

## 1. Flow 5: Full Flow
- **Score**: 222.57
- **Steps**: Noise Reduction, Histogram Equalization, Binarization, Morphological Operations, Edge Detection.
- **Interpretation**:
  - This flow produced the **highest average score**, indicating that combining **all five techniques** resulted in the best improvement in image quality metrics compared to the original images.
  - Including **Edge Detection** along with the previous steps significantly enhanced the clarity and structure of the images, making them more suitable for further deep learning tasks.
  - This suggests that a complete image processing pipeline is necessary to fully prepare the images for accurate table and content recognition.

## 2. Flow 3: Noise Reduction + Histogram Equalization + Binarization
- **Score**: 160.82
- **Steps**: Noise Reduction, Histogram Equalization, Binarization.
- **Interpretation**:
  - This flow performed well, achieving the **second-highest average score**.
  - **Adding Binarization** significantly enhanced the quality compared to earlier steps, making the text and table lines stand out more effectively against the background.
  - However, the absence of **Morphological Operations** and **Edge Detection** meant that it lacked some fine-tuning and edge enhancement, which may have contributed to a slightly lower score than Flow 5.

## 3. Flow 4: Noise Reduction + Histogram Equalization + Binarization + Morphological Operations
- **Score**: 82.40
- **Steps**: Noise Reduction, Histogram Equalization, Binarization, Morphological Operations.
- **Interpretation**:
  - Adding **Morphological Operations** after binarization provided moderate improvements over the earlier stages, but not as much as adding **Edge Detection**.
  - The score is higher compared to the simpler flows, indicating the benefit of **structure-enhancing** techniques like erosion. However, the flow still lacked the fine edge enhancement that Flow 5 achieved.
  - The **drop** in score compared to Flow 3 suggests that Morphological Operations might have introduced some level of noise or data loss that affected the overall clarity.

## 4. Flow 2: Noise Reduction + Histogram Equalization
- **Score**: 4.07
- **Steps**: Noise Reduction, Histogram Equalization.
- **Interpretation**:
  - Adding **Histogram Equalization** improved the results compared to just noise reduction, indicating the importance of adjusting contrast to make text and lines stand out.
  - However, without **binarization** or **edge detection**, this flow was still quite limited in improving the key features needed for better content recognition.

## 5. Flow 1: Noise Reduction
- **Score**: -0.29
- **Steps**: Noise Reduction.
- **Interpretation**:
  - This flow produced a **negative score**, meaning that **only reducing noise** without further enhancements made the images worse in terms of metrics like contrast and sharpness.
  - Noise reduction alone smoothed out the images, but without improving contrast or emphasizing text and lines, the processed images lost significant information compared to the originals.

## 6. Flow - CLAHE
- **Score**: -1.88
- **Steps**: CLAHE.
- **Interpretation**:
  - This flow produced a **negative score**, indicating that applying **CLAHE** alone did not effectively enhance the images.
  - The results suggest that **contrast enhancement** without additional noise reduction or edge detection was not sufficient to improve the overall image quality.

## Summary of Findings
- **Flow 5: Full Flow** achieved the best result, suggesting that the combination of **all five steps**—Noise Reduction, Histogram Equalization, Binarization, Morphological Operations, and Edge Detection—was the most effective in enhancing image quality and making the images suitable for deep learning processes.
- **Flow 3** also performed well, showing that **Binarization** played a crucial role in improving the images by making text and table lines more distinct.
- Adding **Morphological Operations** as in **Flow 4** improved the results somewhat, but it appears that the addition of **Edge Detection** was crucial for getting the highest scores.
- The **best flows** involve a balance of reducing noise, enhancing contrast, clearly separating text from the background, refining the structure, and emphasizing edges.

Based on these results, you can conclude that a comprehensive image processing pipeline involving all the steps yields the best preparation for deep learning tasks, but simpler versions of the flow can still yield reasonable improvements depending on the requirements of the task.
